# Wrangling and Visualizing Rain Guage Data from the Community Collaborative Rain Hail and Snow Network (CoCoRaHS)

[CoCoRAHS](https://www.cocorahs.org) is a network of volunteer precipitation observers that covers the US and Canada. Observers report in daily precipitation totals every day at 7 AM or extreme events such as hail or heavy rain over a short period of time. The data is used by groups such as the National Oceanic and Atmospheric Administration and National Weather Service for forecast decision making and research

Back in high school I was an observer in this network and took various observations. However the website for viewing data and submitting observations was severely limited, and only gave relatively basic options on how to view the data and to download data that could be formatted into software like Microsoft Excel. This was extremely frustating as I always wanted to explore the data I was gather in further depth to find additional trends that weren't displayed or provided by the CoCoRAHs site.

## Enter Python & Data Science

After working in university and industry, I learned that both the Data Science & Data Analysis process alongside _Python_ can be used to both gather CoCoRaHS data into a format that can be more flexibly analyzed and create the visualizations that I've was always wanted to make for CoCoRaHS data. The following notebook is designed to show those techniques and make them accessible to the borader data science community.

## Downloading the Data